# Using Opik with OpenAI

Opik integrates with OpenAI to provide a simple way to log traces for all OpenAI LLM calls. This works for all OpenAI models, including if you are using the streaming API.


## Creating an account on Comet.com

[Comet](https://www.comet.com/site?from=llm&utm_source=opik&utm_medium=colab&utm_content=openai&utm_campaign=opik) provides a hosted version of the Opik platform, [simply create an account](https://www.comet.com/signup?from=llm&utm_source=opik&utm_medium=colab&utm_content=openai&utm_campaign=opik) and grab you API Key.

> You can also run the Opik platform locally, see the [installation guide](https://www.comet.com/docs/opik/self-host/overview/?from=llm&utm_source=opik&utm_medium=colab&utm_content=openai&utm_campaign=opik) for more information.

In [ ]:
%pip install --upgrade opik boto3

In [ ]:
import opik

opik.configure(use_local=False)

## Preparing our environment

First, we will set up our bedrock client. Uncomment the following lines to pass AWS Credentials manually or [checkout other ways of passing credentials to Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html). You will also need to request access to the model in the UI before being able to generate text, here we are gonna use the Amazon titan model, you can request access to it in [this page for the us-east1](https://us-east-1.console.aws.amazon.com/bedrock/home?region=us-east-1#/providers?model=amazon.titan-text-lite-v1) region.

In [ ]:
import boto3

REGION = "us-east-1"

MODEL_ID = "amazon.titan-text-lite-v1"

bedrock = boto3.client(
    service_name="bedrock-runtime",
    region_name=REGION,
    # aws_access_key_id=ACCESS_KEY,
    # aws_secret_access_key=SECRET_KEY,
    # aws_session_token=SESSION_TOKEN,
)

## Logging traces

In order to log traces to Opik, we need to wrap our Bedrock calls with the `track_bedrock` function:

In [ ]:
import os

from opik.integrations.bedrock import track_bedrock

os.environ["OPIK_PROJECT_NAME"] = "bedrock-integration-demo"
bedrock_client = track_bedrock(bedrock)

In [ ]:
PROMPT = "Why is it important to use a LLM Monitoring like CometML Opik tool that allows you to log traces and spans when working with LLM Models hosted on AWS Bedrock?"

response = bedrock_client.converse(
    modelId=MODEL_ID,
    messages=[{"role": "user", "content": [{"text": PROMPT}]}],
    inferenceConfig={"temperature": 0.5, "maxTokens": 512, "topP": 0.9},
)
print("Response", response["output"]["message"]["content"][0]["text"])

The prompt and response messages are automatically logged to Opik and can be viewed in the UI.

![Bedrock Integration](TODO)

## 

## Using it with the `track` decorator

If you have multiple steps in your LLM pipeline, you can use the `track` decorator to log the traces for each step. If Bedrock is called within one of these steps, the LLM call with be associated with that corresponding step:

In [ ]:
from opik import track

os.environ["OPIK_PROJECT_NAME"] = "openai-integration-demo"

bedrock = boto3.client(
    service_name="bedrock-runtime",
    region_name=REGION,
    # aws_access_key_id=ACCESS_KEY,
    # aws_secret_access_key=SECRET_KEY,
    # aws_session_token=SESSION_TOKEN,
)
bedrock_client = track_bedrock(bedrock)


@track
def generate_story(prompt):
    res = bedrock_client.converse(
        modelId=MODEL_ID, messages=[{"role": "user", "content": [{"text": prompt}]}]
    )
    return res["output"]["message"]["content"][0]["text"]


@track
def generate_topic():
    prompt = "Generate a topic for a story about Opik."
    res = bedrock_client.converse(
        modelId=MODEL_ID, messages=[{"role": "user", "content": [{"text": prompt}]}]
    )
    return res["output"]["message"]["content"][0]["text"]


@track
def generate_opik_story():
    topic = generate_topic()
    story = generate_story(topic)
    return story


generate_opik_story()

The trace can now be viewed in the UI:

![Bedrock Integration](TODO)